# RNN Language Model

### Data Things
We're using SST once again. Load the data!

In [9]:
sst_home = '../data/trees'

import re
import random

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

To simplify your implementation, we'll use a fixed unrolling length of 20. This means that we'll have to expand each sentence into a sequence of 21 word indices. In the conversion process, we'll mark the start of each sentence with a special word symbol `<S>`, mark the end of each sentence (if it occurs within the first 21 words) with a special word symbol `</S>`, mark extra tokens after `</S>` with a special word symbol `<PAD>`, and mark out-of-vocabulary words with `<UNK>`, for unknown. As in the previous assignment, we'll use a very small vocabulary for this assignment, so you'll see `<UNK>` often.

In [182]:
import collections
import torch
import numpy as np

max_seq_length = 20

def sentence_to_padded_index_sequence(datasets):
    '''Annotates datasets with feature vectors.'''
    
    START = "<S>"
    END = "</S>"
    END_PADDING = "<PAD>"
    UNKNOWN = "<UNK>"
    SEQ_LEN = max_seq_length + 1
    
    # Extract vocabulary
    def tokenize(string):
        return string.lower().split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    # Only enter word into vocabulary if it appears > 25 times. Add special symbols to vocabulary.
    vocabulary = set([word for word in word_counter if word_counter[word] > 25]) 
    vocabulary = list(vocabulary)
    vocabulary = [START, END, END_PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))
    indices_to_words = {v: k for k, v in word_indices.items()}
        
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['index_sequence'] = torch.zeros((SEQ_LEN))
            
            token_sequence = [START] + tokenize(example['text']) + [END]

            for i in range(SEQ_LEN):
                if i < len(token_sequence):
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                else:
                    index = word_indices[END_PADDING]
                example['index_sequence'][i] = index
                
            example['target_sequence'] = example["index_sequence"][1:]
            example['index_sequence'] = example['index_sequence'].long().view(1,-1)
            example['target_sequence'] = example['target_sequence'].long().view(1,-1)
            
    return indices_to_words, word_indices
    
indices_to_words, word_indices = sentence_to_padded_index_sequence([training_set, dev_set, test_set])

Let's batchify data and define the evaluation metric,

In [183]:
# This is the iterator we'll use during training. 
# It's a generator that gives you one batch at a time.
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        yield [source[index] for index in batch_indices]

# This is the iterator we use when we're evaluating our model. 
# It gives a list of batches that you can then iterate through.
def eval_iter(source, batch_size):
    batches = []
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        if len(batch) == batch_size:
            batches.append(batch)
        else:
            continue
        
    return batches

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    targets = []
    for d in batch:
        vectors.append(d["index_sequence"])
        targets.append(d["target_sequence"])
    return vectors, targets


### Model time!

Now, using the starter code and hyperparameter values provided below, implement an LSTM language model with dropout on the non-recurrent connections. `lstm_step` is provided for you, you need to complete the forward function with a decoder. 

We won't be evaluating our model in the conventional way (perplexity on a held-out test set) for a few reasons: to save time, because we have no baseline to compare against, and because overfitting the training set is a less immediate concern with these models than it was with sentence classifiers. Instead, we'll use the value of the loss function to make sure that the model is converging as expected, and we'll use samples drawn from the model to qualitatively evaluate it.

So you will also need to complete the `sample` function such that when it is called, and it is provided with the starting symbol `<S>`, then is generates a sentence that is 20 tokens long (20 being our max sequence length). You should calucalte word weights with a softmax but make sure to use the softmax trick of subtracting the maximum value,
$$ weight_i = \frac{\exp(x_i - \max_i(x))}{\sum_i(\exp(x_i - max_i(x)))} $$
Then use these weights to sample from the distribution and generate tokens.

Hint: you will need to use `torch.multinomial()` 

In [184]:
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNNLM(nn.Module):
    def __init__(self, vocab_size, seq_len, embedding_dim, hidden_size, batch_size):
        super(RNNLM, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_size = embedding_dim
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        
        self.linear_f = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear_i = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear_ctilde = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear_o = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(p=0.25)
        
        self.init_weights()
                
    def forward(self, x, hidden, c):
        x_emb = self.encoder(x)
        embs = torch.chunk(x_emb, x_emb.size()[1], 1)  
        hiddens = []
        logits = []
        for i in range(self.seq_len - 1):
            hidden, c = self.lstm_step(embs[i].squeeze(), hidden, c)
            hiddens.append(hidden)
            hidden_drop = self.dropout(hidden)
            logit = self.decoder(hidden_drop)
            logits.append(logit)
            
        return hiddens, logits  
        
    def sample(self, x_start, hidden, c):
        indices = [int(x_start.data.numpy())]
        for i in range(self.seq_len - 1):
            x_emb = self.encoder(x_start)
            hidden, c = self.lstm_step(x_emb.view(1,-1), hidden, c) 
            logits = self.decoder(hidden)
            
            logits = logits.squeeze().data
            exp_logits = torch.exp(logits - torch.max(logits))
            word_weights = exp_logits / exp_logits.sum()
            word_idx = torch.multinomial(word_weights, 1)[0]
            x_start.data.fill_(word_idx)
            indices.append(word_idx)
            
        return indices
        
    def lstm_step(self, emb, hid, c_t):
        emb_drop = self.dropout(emb)
        combined = torch.cat((hid, emb_drop), 1)
        f = F.sigmoid(self.linear_f(combined))
        i = F.sigmoid(self.linear_i(combined))
        c_tilde = F.tanh(self.linear_ctilde(combined))
        c_t = f * c_t + i * c_tilde
        o = F.sigmoid(self.linear_o(combined))
        hid = o * F.tanh(c_t)
        return hid, c_t
    
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.linear_f, self.linear_i, self.linear_ctilde, self.linear_o, self.decoder]
        em_layer = [self.encoder]
     
        for layer in lin_layers+em_layer:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)
                
    def init_hidden(self, bsz):
        h0 = Variable(torch.zeros(bsz, self.hidden_size))
        c0 = Variable(torch.zeros(bsz, self.hidden_size))
        return h0, c0

### Define the training loop,

In [185]:
def training_loop(batch_size, vocab_size, num_epochs, display_freq, model, criterion, optim, training_iter):
    step = 1
    epoch = 1
    total_batches = int(len(training_set) / batch_size)

    while epoch <= num_epochs:
        model.train()      
        inputs, targets = get_batch(next(training_iter)) 
        inputs = Variable(torch.stack(inputs).squeeze(), volatile=False) # batch_size,  seq_len
        targets = Variable(torch.stack(targets).squeeze().view(-1)) # batch_size,seq_len --> flat, b*s
    
        model.zero_grad()     
        hidden, c_t = model.init_hidden(batch_size)
        hiddens, output = model(inputs, hidden, c_t)
        output_flat = torch.stack(output, dim=1).view(-1, vocab_size) # batch_size, seq_len, hid --> b*s,h

        lossy = criterion(output_flat, targets)
        lossy.backward()
        optim.step()
        
        if step % total_batches == 0:
            if epoch % display_freq == 0:
                hidden, c_t = model.init_hidden(1)
                input = Variable(torch.zeros(1, 1).long(), volatile=True)
                indices = model.sample(input, hidden, c_t)
                words = [indices_to_words[index] for index in indices]
                print("Epoch: {}; Loss: {}; Sample: {}".format(epoch, lossy.data[0], ' '.join(words)))
            epoch += 1
        step += 1

### Train and sample!

Now train model with the suggested hyperparameters.

Once you're confident your model is doing what you want, let it run for the full 350 epochs. This will take some time—likely between ten and forty minutes, so do it at home if you don't have time during the class. We train the model for a fairly long time because these small improvements in cost correspond to fairly large improvements in sample quality.

Samples from a trained models should have coherent portions, but they will not resemble interpretable English sentences.

In [186]:
# Hyper Parameters 
vocab_size = len(indices_to_words)
seq_length = max_seq_length + 1
hidden_dim = 32
embedding_dim = 16
batch_size = 256
learning_rate = 0.3
num_epochs = 350
display_epoch_freq = 1

# Build, initialize, and train model
lm = RNNLM(vocab_size, seq_length, embedding_dim, hidden_dim, batch_size)
lm.init_weights()

# Loss and Optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(lm.parameters(), lr=learning_rate)

# Train the model
training_iter = data_iter(training_set, batch_size)
training_loop(batch_size, vocab_size, num_epochs, display_epoch_freq, lm, loss, optimizer, training_iter)

Epoch: 1; Loss: 5.46087121963501; Sample: <S> nearly told fascinating making lack shows silly films always ride gets history were lacks gags no feature enjoyable those overall
Epoch: 2; Loss: 4.456007480621338; Sample: <S> looking <UNK> <UNK> <UNK> us personal film me completely <PAD> summer too goes 'd <PAD> michael acting off <UNK> <PAD>
Epoch: 3; Loss: 4.263888835906982; Sample: <S> as <UNK> humor actually you come <PAD> violence <UNK> <PAD> comes <UNK> <UNK> role <PAD> <PAD> <PAD> <PAD> <UNK> <PAD>
Epoch: 4; Loss: 3.9805500507354736; Sample: <S> none without little charming characters wrong <PAD> <PAD> interest <PAD> <UNK> intelligent sure <UNK> <PAD> <UNK> <UNK> <PAD> <PAD> <UNK>
Epoch: 5; Loss: 3.738694429397583; Sample: <S> <PAD> mr. <PAD> the minutes study <PAD> <PAD> character a engaging <PAD> the <PAD> a , <PAD> <PAD> <PAD> <PAD>
Epoch: 6; Loss: 3.5647494792938232; Sample: <S> play can whole is sex <PAD> , <PAD> <UNK> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

Epoch: 55; Loss: 3.1482837200164795; Sample: <S> <UNK> <UNK> <UNK> narrative the <UNK> minutes here terrific <UNK> as <UNK> <UNK> of making <UNK> . </S> <PAD> <PAD>
Epoch: 56; Loss: 3.1715989112854004; Sample: <S> to a in <UNK> will a an <UNK> -rrb- watching is fun has character <UNK> of going and <UNK> himself
Epoch: 57; Loss: 3.211613178253174; Sample: <S> impossible power <UNK> but <UNK> <UNK> all crime almost well comedy like <UNK> is every <UNK> <UNK> turn likely you
Epoch: 58; Loss: 3.1630446910858154; Sample: <S> days <UNK> <UNK> <UNK> <UNK> but <UNK> <UNK> of those and of the <UNK> <UNK> 's <UNK> is <UNK> <UNK>
Epoch: 59; Loss: 3.154677152633667; Sample: <S> and <UNK> new `` <UNK> <UNK> is <UNK> 's <UNK> any : in <UNK> might his <UNK> . </S> <PAD>
Epoch: 60; Loss: 3.171872854232788; Sample: <S> a about <UNK> melodrama <UNK> the film <UNK> <UNK> it french images it be the <UNK> <UNK> <UNK> <UNK> ,
Epoch: 61; Loss: 3.0503907203674316; Sample: <S> you than about its <UNK> n't can 

Epoch: 110; Loss: 3.104783058166504; Sample: <S> <UNK> fresh '' <UNK> modern <UNK> and <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 111; Loss: 3.021033763885498; Sample: <S> yet a <UNK> 's short ... . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 112; Loss: 3.0972237586975098; Sample: <S> black funny is getting one . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 113; Loss: 2.9712319374084473; Sample: <S> director women some with <UNK> <UNK> <UNK> and anyone <UNK> here <UNK> for as the no has <UNK> <UNK> of
Epoch: 114; Loss: 2.9188807010650635; Sample: <S> remains yet the <UNK> -rrb- <UNK> <UNK> that <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 115; Loss: 2.947082042694092; Sample: <S> they it 's <UNK> look a <UNK> <UNK> to <UNK> <UNK> to <UNK> <UNK> left <UNK> <UNK> on the <UNK>
Epoch: 116; Loss: 2.993809700012207; Sample: <S> know do <UNK> t

Epoch: 165; Loss: 2.90327787399292; Sample: <S> this just comedy <UNK> fact and <UNK> a <UNK> a film like ride to <UNK> as be <UNK> in the
Epoch: 166; Loss: 2.9766788482666016; Sample: <S> study these thing and its <UNK> <UNK> and history . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 167; Loss: 3.18192458152771; Sample: <S> probably <UNK> high compelling perfectly the <UNK> <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 168; Loss: 2.8918492794036865; Sample: <S> <UNK> she of <UNK> an no to the film <UNK> 's <UNK> <UNK> he , <UNK> yet an <UNK> drama
Epoch: 169; Loss: 3.0746452808380127; Sample: <S> a fascinating <UNK> will out and <UNK> , at <UNK> of the <UNK> , <UNK> who <UNK> <UNK> . </S>
Epoch: 170; Loss: 3.0403997898101807; Sample: <S> nothing enjoy so <UNK> : more movie to message <UNK> into <UNK> 's <UNK> left , the <UNK> of <UNK>
Epoch: 171; Loss: 2.940624952316284; Sample: <S> <UNK> gets . or opera . </S> <PAD> <PAD> <PAD> <

Epoch: 219; Loss: 2.964733362197876; Sample: <S> filmmaker too could dramatic <UNK> that <UNK> 's we comes by none of the <UNK> during <UNK> . </S> <PAD>
Epoch: 220; Loss: 2.8489484786987305; Sample: <S> a <UNK> that <UNK> <UNK> <UNK> as if us <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 221; Loss: 3.0403082370758057; Sample: <S> sure been cast <UNK> that want to be the <UNK> and <UNK> he </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 222; Loss: 2.8653080463409424; Sample: <S> a <UNK> the who if <UNK> director scenes , a <UNK> in <UNK> , 's video <UNK> that years it
Epoch: 223; Loss: 2.8731276988983154; Sample: <S> and action <UNK> <UNK> in a <UNK> more <UNK> <UNK> <UNK> <UNK> '' into <UNK> <UNK> film have tries would
Epoch: 224; Loss: 2.9928526878356934; Sample: <S> the people <UNK> as the <UNK> are of <UNK> <UNK> and <UNK> <UNK> of the <UNK> <UNK> <UNK> to <UNK>
Epoch: 225; Loss: 2.856098175048828; Sample: <S> not <UNK> into its <UNK> thought with better comic

Epoch: 274; Loss: 2.8405888080596924; Sample: <S> n't <UNK> coming-of-age <UNK> with see interesting to new film is n't of still <UNK> <UNK> to his very <UNK>
Epoch: 275; Loss: 2.9528439044952393; Sample: <S> <UNK> so <UNK> , the role of <UNK> to make a <UNK> 's the thought <UNK> <UNK> does its its
Epoch: 276; Loss: 2.828068494796753; Sample: <S> <UNK> has have <UNK> is by <UNK> has <UNK> a <UNK> into the <UNK> <UNK> could <UNK> over as music
Epoch: 277; Loss: 2.854419708251953; Sample: <S> <UNK> <UNK> <UNK> in <UNK> , the <UNK> for the smart of <UNK> <UNK> to <UNK> dumb . </S> <PAD>
Epoch: 278; Loss: 2.9468915462493896; Sample: <S> seems narrative the <UNK> ... directed <UNK> <UNK> through <UNK> <UNK> enjoyable of the <UNK> show of <UNK> and <UNK>
Epoch: 279; Loss: 2.923605442047119; Sample: <S> <UNK> , there 's time is a <UNK> of the <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 280; Loss: 3.0577080249786377; Sample: <S> <UNK> <UNK> <UNK> funny in your <UNK> of <UNK> 

Epoch: 329; Loss: 2.948920488357544; Sample: <S> if it is about <UNK> should have what in <UNK> <UNK> to <UNK> over on the whole <UNK> most <UNK>
Epoch: 330; Loss: 2.873727321624756; Sample: <S> this <UNK> <UNK> , but <UNK> <UNK> 's <UNK> <UNK> of the movie that good <UNK> <UNK> <UNK> and a
Epoch: 331; Loss: 2.897233009338379; Sample: <S> about but i a <UNK> of is he make quite a ends of those <UNK> comic but <UNK> from <UNK>
Epoch: 332; Loss: 2.9485654830932617; Sample: <S> <UNK> <UNK> rather <UNK> <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 333; Loss: 3.021697521209717; Sample: <S> and its <UNK> ` <UNK> for a <UNK> audiences <UNK> with very <UNK> french how <UNK> amusing to new sense
Epoch: 334; Loss: 3.011765956878662; Sample: <S> <UNK> true viewers a <UNK> of <UNK> . </S> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Epoch: 335; Loss: 2.9130468368530273; Sample: <S> . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

## Questions:

**Question 1:** Looking at the samples that your model produced towards the end of training, what properties of (written) English does it seem to have learned.

**Question 2:** If we could make the model as big as we wanted, train as long as we wanted, and adjust or remove dropout at will, could we ever get the model to reach a cost value of 0.0?

**Question 3:** Would the model be any worse if we were to just delete unknown words instead of using an `<UNK>` token?

## Extras:

* Do weight tying, i.e. reuse the input word embedding matrix as the output classification matrix. _https://arxiv.org/pdf/1611.01462.pdf_
* Implement beam search decoding